# City Schools Analysis

**By:** Tania Barrera (*tsbarr*)

This Jupyter Notebook goes over my analysis of city schools data for Challenge 4 of the UofT SCS EdX Data Bootcamp, using the python module `pandas`.

It includes a summary of the whole district and of each school, including displaying the highest and lowest performing schools, as well as math and reading grades analysis by grade, school spending, school size and school type.



---

## Table of Contents

- [Initial Setup](#initial-setup)
  - [Imports](#imports)
  - [Input data](#input-data)
  - [Helper columns](#helper-columns)
- [Data analysis](#data-analysis)
  - [District Summary](#district-summary)
  - [School Summary](#school-summary)
  - [Highest-Performing Schools (by % Overall Passing)](#highest-performing-schools-by--overall-passing)
  - [Lowest-Performing Schools (by % Overall Passing)](#lowest-performing-schools-by--overall-passing)
  - [Math Scores by Grade](#math-scores-by-grade)
  - [Reading Scores by Grade](#reading-scores-by-grade)
  - [Scores by School Spending](#scores-by-school-spending)
  - [Scores by School Size](#scores-by-school-size)
  - [Scores by School Type](#scores-by-school-type)
- [Conclusions](#conclusions)
- [References](#references)

---



## Initial Setup

### Imports

The first step before performing any analysis is importing the necessary modules.

The imports I am using for this project are:

- Module **`pandas`**: to perform dataframe analysis
- Subclass **`Path`** from the **`pathlib`** module: to create the file path object that is used to read in data.
- Subclass CategoricalDtype from pandas.api.types to make grade a cateogrical ordered variable when analysing scores by grade.


In [1]:
# Import modules
import pandas as pd
from pandas.api.types import CategoricalDtype
from pathlib import Path

### Input data

There are two datasets that are imported for this project: School Data and Student Data.

The `school_data` dataframe has the columns:

- `School ID`: unique id number as an integer, starting from 0.
- `school_name`: the name of this school
- `type`: can be District or Charter
- `size`: number of students in this school
- `budget`: budget of this school

And the `student_data` dataframe has the columns:

- `Student ID`: unique id number as an integer, starting from 0
- `student_name`: name of this student
- `gender`: F for female or M for male
- `grade`: as string representing an ordinal: 9th, 10th, 11th or 12th
- `school_name`: what school the student is in, should correspond to one of the values in column school_name of the school dataset
- `reading_score`: an integer up to 100
- `reading_score`: an integer up to 100

I left joined these two dataframes on school_name into the dataframe `all_data`.

In [2]:
# Input file paths
school_input_path = Path("Resources/schools_complete.csv")
student_input_path = Path("Resources/students_complete.csv")

# Read School and Student Data and store into Pandas DataFrames
school_data = pd.read_csv(school_input_path)
student_data = pd.read_csv(student_input_path)

# Combine the data into a single dataset.
# from guide: https://pandas.pydata.org/pandas-docs/version/0.24.0/user_guide/merging.html
all_data = school_data\
    .merge(student_data, how='left', on='school_name')

# visualize first rows of combined data set
all_data.head()


,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


### Helper columns

Since several of the requested summaries need me to calculate the percentage of students that passed math, reading or both over different grouping variables, I generate 3 extra boolean columns that indicate if the student passed to help me calculate these.

In [3]:
# create new columns indicating if the student passed math and reading,
# based on: https://sparkbyexamples.com/pandas/pandas-add-column-based-on-another-column/

all_data['passed_math'] = all_data['math_score'] >= 70
all_data['passed_reading'] = all_data['reading_score'] >= 70

# use these new columns to add another column showing if they passed both
all_data['passed_overall'] = (all_data['passed_math'] & all_data['passed_reading'])

# visualize first rows
all_data.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score,passed_math,passed_reading,passed_overall
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,True,False,False
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,False,True,False
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,False,True,False
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,False,False,False
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,True,True,True


## Data Analysis

### District Summary

The first analysis is a high-level snapshot of the district's key metrics, which is stored in a DataFrame.

The columns in this summary are calculated as follows:

- `Total Schools`: use the `school_data` dataframe and count the unique school IDs using the `.nunique()` method on the `School ID` column.

- `Total Students`: use the `student_data` data frame and count the unique student IDs using the `.nunique()` method on the `Student ID` column.

- `Total Budget`: use the `school_data` dataframe and the `.sum()` method on the budget column.

- `Average Math Score`: use the `student_data` dataframe and the `.mean()` method on the `math_score` column.

- `Average Reading Score`: use the `student_data` dataframe and the `.mean()` method on the `reading_score` column.

- `% Passing Math`: use the `all_data` dataframe, which includes the previously generated column `passed_math`. This column is a boolean and using the `.sum()` method will return the number of students that passed math. Then, divide this count by the number of total students, calculated as decribed above, and finally, multiply by 100 to get the percentage.

- `% Passing Reading`: similar to `% Passing Math`, but using the `passed_reading` column instead of `passed_math`.

- `% Passing Overall`: similar to `% Passing Math`, but using the `passed_overall` column instead of `passed_math`.

In [4]:
# Calculate summary metrics and store in dataframe district_summary

district_summary = pd.DataFrame(
    {
        'Total Schools': [school_data['School ID'].nunique()],
        'Total Students': [student_data['Student ID'].nunique()],
        'Total Budget' : [school_data['budget'].sum()],
        'Average Math Score': [student_data['math_score'].mean()],
        'Average Reading Score': [student_data['reading_score'].mean()],
        '% Passing Math': [all_data['passed_math'].sum() / student_data['Student ID'].nunique() * 100],
        '% Passing Reading': [all_data['passed_reading'].sum() / student_data['Student ID'].nunique() * 100],
        '% Passing Overall': [all_data['passed_overall'].sum() / student_data['Student ID'].nunique() * 100]
    }
)

# visualize summary dataframe
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


### School Summary

Some of the requested fields for the `per_school_summary` dataframe do not have to be calculated, since they are part of the `school_data` dataframe: school name, school type, total students and total school budget. 

The rest of the columns are calculated from the `all_data` dataframe, grouping by the columns that contain the school data we want to keep.

After grouping, I use the `.aggregate()` function to calculate the mean math and reading scores and the number of students that passed math, reading and both by summing the previously generated columns `passed_math`, `passed_reading` and `passed_overall`.

This results in a new dataframe that is aggregated by school. Then, I use the `.assign()` method to generate the column for the budget per student (budget of the school divided by number of students) and the percent of students that passed math, reading and both in each school (count of students that passed each, divided by total students and multiplied by 100).

I then drop the columns that have the count of students that passed math, reading and overall, since the percents were already calculated and we don't need those columns.

Finally, I clean the column names by renaming some with more descriptive names, switching the word `percent` for the `%` symbol and the underscores for spaces and making all the names title-case.

In [5]:
# create school summary based on all_data dataframe
per_school_summary = (all_data
    # first, group all data by school, and the other columns that contain the school data we want to keep
    .groupby(['school_name', 'type', 'size', 'budget'], as_index=False)
    # use aggregate to get average of math and reading scores by school
    # based on:
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html
    # https://stackoverflow.com/a/43897124
    .aggregate(
        average_math_score = ('math_score', 'mean'),
        average_reading_score = ('reading_score', 'mean'),
        # count students that passed math, reading and overall (to calc % later)
        count_pass_math = ('passed_math', 'sum'),
        count_pass_reading = ('passed_reading', 'sum'),
        count_pass_overall = ('passed_overall', 'sum')
    )
    # use assign to calculate new columns of budget per student and percent passing
    # based on:
    # https://sparkbyexamples.com/pandas/pandas-add-column-based-on-another-column/
    .assign(
        per_student_budget = lambda df: 
            df['budget'] / df['size'],
        percent_passing_math = lambda df:
            df['count_pass_math'] / df['size'] * 100,
        percent_passing_reading = lambda df:
            df['count_pass_reading'] / df['size'] * 100,
        percent_passing_overall = lambda df:
            df['count_pass_overall'] / df['size'] * 100
    )
    # drop columns with counts that we don't need anymore
    .drop(columns = ['count_pass_math', 'count_pass_reading', 'count_pass_overall'])
    # rename columns with more descriptive names
    .rename(
        columns = {
            'type' : 'School type',
            'size' : 'Total students',
            'budget' : 'Total school budget'
        }
    )
    # rename columns to replace 'percent' with '%', '_' with ' ' and make them all title case
    # based on:
    # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
    .rename(
        columns = lambda x: 
        (x.replace('percent', '%')
            .replace('_', ' ')
            .title()
        )
    )
)

# visualize summary dataframe
per_school_summary

,School Name,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Passing Overall
0,Bailey High School,District,4976,3124928,77.048432,81.033963,628.0,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,53.204476
3,Ford High School,District,2739,1763916,77.102592,80.746258,644.0,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,248087,83.803279,83.814988,581.0,92.505855,96.252927,89.227166
7,Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,585858,83.839917,84.044699,609.0,94.594595,95.945946,90.540541


### Highest-Performing Schools (by % Overall Passing)

I generate a new dataframe only containing the top 5 schools with the highest percentage of passing both math and reading (overall), by using the `.nlargest()` method in the `per_school_summary `dataframe, with arguments `n = 5 `and `columns = '% Passing Overall'`. Afterwards I reset the index to make it look nicer.

In [6]:
# use nlargest to get top 5 schools by % passing overall
top_schools = (per_school_summary
    .nlargest(5, '% Passing Overall')
    .reset_index(drop = True)
)
# visualize results
top_schools

,School Name,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Passing Overall
0,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635,1043130,83.418349,83.848930,638.0,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283,1319574,83.274201,83.989488,578.0,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,585858,83.839917,84.044699,609.0,94.594595,95.945946,90.540541


### Lowest-Performing Schools (by % Overall Passing)

I generate a new dataframe only containing the bottom 5 schools with the lowest percentage of passing both math and reading (overall), by using the `.nsmallest()` method in the `per_school_summary `dataframe, with arguments `n = 5 `and `columns = '% Passing Overall'`. Afterwards I reset the index to make it look nicer.

In [7]:
# use nsmallest to get bottom 5 schools by % passing overall
bottom_schools = (per_school_summary
    .nsmallest(5, '% Passing Overall')
    .reset_index(drop = True)
)
# visualize results
bottom_schools

,School Name,School Type,Total Students,Total School Budget,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Passing Overall
0,Rodriguez High School,District,3999,2547363,76.842711,80.744686,637.0,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,53.204476
2,Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0,66.752967,80.862999,53.527508
4,Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,53.539172


### Math Scores by Grade

To obtain the math scores by grade, I first convert the `grade` column of the `all_data` dataframe into an ordered categorical variable. This way, once I get the results, they will be sorted in order with 9th graders first and 12th graders last (if it stayed as string type, the string alphabetical sorting would put 10th first and 9th last).

Then, I use the `.pivot_table()` method which first groups the data and then generates aggregated summary columns per group. This method takes arguments so we can tell it how to group (`index='school_name'`, `columns='grade'`) and what to use to aggregate (`values='math_score'`, `aggfunc='mean'`).

The resulting dataframe has the `school_name` column as row index and names the column index as 'grade', so I reset the index without dropping it and using the argument `names` to rename the former index column to `School Name` and get rid of the column index name using `.columns.name = None`.


In [8]:
# make 'grade' a categorical ordered variable
all_data['grade'] = all_data['grade'].astype(CategoricalDtype(categories=['9th', '10th', '11th', '12th'], ordered=True))

# create a pivot table based on the all_data dataframe
# based on:
# https://pandas.pydata.org/docs/dev/user_guide/reshaping.html
# https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html
math_scores_by_grade = (all_data
    .pivot_table(
        index='school_name',
        columns='grade',
        values='math_score',
        aggfunc='mean'
    )
    .reset_index(drop=False, names='School Name')
)
# fix column index name
math_scores_by_grade.columns.name = None
# visualize results
math_scores_by_grade


,School Name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


### Reading Scores by Grade

The process to obtain the reasing scores by grade is similar to the above process for math scores by grade, except I had already converted the `grade` column to a cataegorical variable so I don't need to do that again. 

I use the `.pivot_table()` method which first groups the data and then generates aggregated summary columns per group. This method takes arguments so we can tell it how to group (`index='school_name'`, `columns='grade'`) and what to use to aggregate (`values='reading_score'`, `aggfunc='mean'`).

The resulting dataframe has the `school_name` column as row index and names the column index as 'grade', so I reset the index without dropping it and using the argument `names` to rename the former index column to `School Name` and get rid of the column index name using `.columns.name = None`.

In [9]:
# create a pivot table based on the all_data dataframe
reading_scores_by_grade = (all_data
    .pivot_table(
        index='school_name',
        columns='grade',
        values='reading_score',
        aggfunc='mean'
    )
    .reset_index(drop=False, names='School Name')
)
# fix column index name
reading_scores_by_grade.columns.name = None
# visualize results
reading_scores_by_grade

,School Name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


### Scores by School Spending

The school performance metrics we are using are mean math and reading score, as well as percentage passing of math, reading and both. To analyze them based on how much schools spend per student, I first need to create a copy of the school summary dataframe, to not edit the original school summary. Then, I use the `.cut()` method to create a new variable that categorizes schools by how much they spend per student and use this new variable to group the data and get the mean of the performance metrics for each category.

In [10]:
# make a copy of the per_school_summary dataframe that will be used for the spending summary
spending_summary = per_school_summary.copy()
# create a new variable based on spending categories
# based on: https://pandas.pydata.org/docs/reference/api/pandas.cut.html
spending_summary['Spending Ranges (Per Student)'] = pd.cut(
    x = spending_summary['Per Student Budget'],
    bins = [0, 585, 630, 645, 680],
    labels = ['<$585', '$585-630', '$630-645', '$645-680'])
# group by the new variable and get mean values for performance metrics
spending_summary = (spending_summary
    .groupby('Spending Ranges (Per Student)', as_index = False)
    .aggregate(
        {
            'Average Math Score' : 'mean',
            'Average Reading Score' : 'mean',
            '% Passing Math' : 'mean',
            '% Passing Reading' : 'mean',
            '% Passing Overall' : 'mean'
        }
    )
)
# visualize results
spending_summary


,Spending Ranges (Per Student),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,<$585,83.455399,83.933814,93.460096,96.610877,90.369459
1,$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
2,$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
3,$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


### Scores by School Size

Similar to how I analyze performance based on spending categories, to analyze performance by school size, I first need to create a copy of the school summary dataframe, to not edit the original school summary. Then, I use the `.cut()` method to create a new variable that categorizes schools by how many students they have and use this new variable to group the data and get the mean of the performance metrics for each category.

In [11]:
# make a copy of the per_school_summary dataframe that will be used for the size summary
size_summary = per_school_summary.copy()
# create a new variable based on size categories
size_summary['School Size'] = pd.cut(
    x = size_summary['Total Students'],
    bins = [0, 1000, 2000, 5000],
    labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"])
# group by the new variable and get mean values for performance metrics
size_summary = (size_summary
    .groupby('School Size', as_index = False)
    .aggregate(
        {
            'Average Math Score': 'mean',
            'Average Reading Score': 'mean',
            '% Passing Math': 'mean',
            '% Passing Reading': 'mean',
            '% Passing Overall': 'mean'
        }
    )
)
# visualize results
size_summary


,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
1,Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
2,Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


### Scores by School Type

To look at performance metrics of schools that are Charter type vs District type, we do not need to create a new variable, since we already have a school type variable. Therefore, we just need to group the school summary by this variable, `School Type` and aggregate using the mean of each performance metric by type of school.

In [12]:
# group by school type and get mean values for performance metrics
type_summary = (per_school_summary
    .groupby('School Type', as_index = False)
    .aggregate(
        {
            'Average Math Score': 'mean',
            'Average Reading Score': 'mean',
            '% Passing Math': 'mean',
            '% Passing Reading': 'mean',
            '% Passing Overall': 'mean'
        }
    )
)
# visualize results
type_summary

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,Charter,83.473852,83.896421,93.620830,96.586489,90.432244
1,District,76.956733,80.966636,66.548453,80.799062,53.672208


## Conclusions

After performing the analysis of the district's schools and the math and reading scores of their students, I can draw several conclusions, including:

1. **District vs Charter:** the school type has an effect on the grades of the students. Charter schools have a higher average score for both math and reading. Also, the percentage of students passing math, reading and both is way higher in charter schools (93.6%, 96.6%, 90.4%, respectively) than in district schools (66.5%, 80.8%, 53.7%, respectively). Another notable detail is that when we look at the performance of the schools by the percentage of students that passed both math and reading, the top 5 schools are all charter schools and the bottom 5 are all district schools.

2. **School size:** we see a big drop in performance in large schools (2000-5000 students) compared to medium (1000-2000 students) and small (less than 1000 students) sized schools. The overall passing percentage in large school is 58.3%, compared to 90.6% in medium schools and 89.9% in small schools. This seems to be mostly affected by math, since the percentage of students that passed this subject is 93.6%, 93.6% and 69.96% in small, medium and large schools respectively, compared to percentage of passing reading which is 96.1%, 96.8% and 82.8% in small, medium and large schools, respectively.

3. **Spending per student:** we also see a drop in overall passing percentage as budget per student increases, with percentage of overall passing going from 90.4% in schools that spend less than $585 per student to 53.5% in schools that spend $645-680 per student.


## References

Data generated by [Mockaroo, LLC](https://mockaroo.com/), (2022). Realistic Data Generator. Data for this dataset was generated by edX Boot Camps LLC, and is intended for educational purposes only.

Challenge instructions and input file, as well as some code sections that were adapted from the UofT SCS EdX Data Bootcamp class activities:

© 2022 edX Boot Camps LLC. Confidential and Proprietary. All Rights Reserved.

Some other code sections were adapted from other sources, specific explanations are found within the code comments:

| Purpose | URL |
| :- | :- |
| Guide to combine the data into a single dataset | https://pandas.pydata.org/pandas-docs/version/0.24.0/user_guide/merging.html |
| How to create new columns based on other columns | https://sparkbyexamples.com/pandas/pandas-add-column-based-on-another-column/ |
| How to use aggregate to create more than one summary column | https://stackoverflow.com/a/43897124 |
| `.aggregate()` documentation | https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.aggregate.html |
| `.rename()` documentation | https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html |
| Guide on reshaping data, including using `.pivot_table()` | https://pandas.pydata.org/docs/dev/user_guide/reshaping.html |
| `.pivot_table()` documentation | https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html |
| `.cut()` documentation | https://pandas.pydata.org/docs/reference/api/pandas.cut.html |

